In [1]:
#!pip install kafka
#!pip install sqlalchemy
#!pip install pandas

In [2]:
#imports
from kafka import KafkaConsumer
from kafka import TopicPartition
from sqlalchemy import create_engine
import pandas as pd
import json

In [3]:
#set parameters
kafka_server = 'localhost:9092'
db_user = 'postgres'
db_password = 'postgres'
db_host = 'localhost'
db_port = '5432'
db_name = 'postgres'

In [4]:
#init consumer
consumer = KafkaConsumer(
    bootstrap_servers=kafka_server,
    auto_offset_reset='earliest',
    value_deserializer=lambda v: json.loads(v.decode('utf-8').replace("'", '"'))
)

In [5]:
#init database
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
try:
    engine.connect()
    print("success")
except SQLAlchemyError as err:
    print("error", err.__cause__)

success


In [6]:
#find current lenght of 'delays' topic
partition_delays = TopicPartition('delays',0)
consumer.assign([partition_delays])

consumer.seek_to_end(partition_delays)
lastOffset = consumer.position(partition_delays)

consumer.seek_to_beginning(partition_delays)

In [7]:
#consume delays
delay_df = pd.DataFrame()

for msg in consumer:
    row=msg.value
    #currently creating a df, TODO: write to db here
    delay_df = pd.concat([delay_df, pd.DataFrame([row])], ignore_index=True)
    if msg.offset == lastOffset - 1:
            break

In [8]:
#find current lenght of 'weather' topic
partition_weather = TopicPartition('weather',0)
consumer.assign([partition_weather])

consumer.seek_to_end(partition_weather)
lastOffset = consumer.position(partition_weather)

consumer.seek_to_beginning(partition_weather)

In [9]:
#consume weather
weather_df = pd.DataFrame()

for msg in consumer:
    row=msg.value
    weather_df = pd.concat([weather_df, pd.DataFrame([row])], ignore_index=True)
    if msg.offset == lastOffset - 1:
            break

In [10]:
consumer.close()

In [11]:
delay_df['start'] = pd.to_datetime(delay_df['start'], format='%d.%m.%Y %H:%M')#
delay_df['end'] = pd.to_datetime(delay_df['end'], format='%d.%m.%Y %H:%M')
delay_df.head()

,id,title,lines,stations,start,end
0,155617,Betrieb bis Emil-Behring-Weg,[63A],[Schloss Hetzendorf],2023-09-19 04:30:00,2024-09-19 23:59:00
1,156071,Betrieb bis Emil-Behring-Weg,[63A],[Schloss Hetzendorf],2023-09-19 04:30:00,2024-09-19 23:59:00
2,83214,Verkehrsbedingte Verspätungen,[49],"[ Bujattigasse, Ameisgasse, Baumgarten, Beinga...",2022-06-30 08:37:00,2022-06-30 23:59:00
3,156080,Betrieb bis Emil-Behring-Weg,[63A],[Schloss Hetzendorf],2023-09-19 04:30:00,2024-09-19 23:59:00
4,83213,52 Verkehrsunfall,[52],"[Ameisgasse, Anschützgasse, Baumgarten, Dieste...",2022-06-30 08:32:00,2022-06-30 09:21:00


In [12]:
delay_df.to_sql('delays', engine, if_exists='append', index=False)

14

In [13]:
weather_df['date'] = pd.to_datetime(weather_df['date'], format='%d/%m/%y')
weather_df.columns = ['date', 'weather_code','temperature','daylight','precipitation','wind']
weather_df.to_sql('weather', engine, if_exists='append', index=True,index_label='id')

472